In [1]:
# Make sure you're on Python > 3.8
# !pip install -r requirements.txt --quiet

In [2]:
from collections import OrderedDict

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import flwr as fl
from flwr.simulation import run_simulation
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents

In [3]:
DEVICE = torch.device('cpu')

## Load and Pre Process Data

In [4]:
!mkdir '.kaggle'
!mkdir '.kaggle/data'

with open(".kaggle/kaggle.json", 'a+') as f:
    f.write('{"username":"rajaxarcmu","key":"68d40c5e38e1c786ab57736bc5c9b2cb"}')
    
!chmod 600 '.kaggle/kaggle.json'
!kaggle datasets download -d 'danofer/compass'
!unzip -qo compass.zip -d '.kaggle/data'

mkdir: .kaggle: File exists
mkdir: .kaggle/data: File exists
Dataset URL: https://www.kaggle.com/datasets/danofer/compass
License(s): DbCL-1.0
compass.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!ls .kaggle/data

compas-scores-raw.csv
cox-violent-parsed.csv
cox-violent-parsed_filt.csv
propublicaCompassRecividism_data_fairml.csv


In [6]:
df = pd.read_csv('.kaggle/data/propublicaCompassRecividism_data_fairml.csv/propublica_data_for_fairml.csv')
print(df.shape)

(6172, 12)


In [7]:
df['caucasian'] = ((df['African_American'] + df['Asian'] + df['Hispanic'] + df['Native_American'] + df['Other']) == 0).astype(int)

In [8]:
NUM_CLIENTS = 10
# REPRESENTS SILO'D ORGANIZATIONS

In [9]:
from datasets import Dataset
from flwr_datasets.partitioner import DirichletPartitioner

In [10]:
trainset, testset = train_test_split(df, test_size=0.2)
batch_size = 32

ds = Dataset.from_pandas(trainset)

partitioner = DirichletPartitioner(
    num_partitions=NUM_CLIENTS,
    partition_by="caucasian",
    alpha=0.5,
    min_partition_size=(len(trainset) // (4 * NUM_CLIENTS)),
    self_balancing=True,
    shuffle=True
)

partitioner.dataset = ds
datasets = []
for i in range(NUM_CLIENTS):
    curr_partition = partitioner.load_partition(i)
    datasets.append(curr_partition.to_pandas())

train_loaders = []
val_loaders = []

feature_columns = ['Number_of_Priors', 'score_factor','Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'Misdemeanor']

for ds in datasets:
    train_x = ds[feature_columns].values
    train_y = ds['Two_yr_Recidivism'].values
    sensitive_feature = ds['caucasian'].values

    train_x, val_x, train_y, val_y, sensitive_train, sensitive_val = train_test_split(
        train_x, train_y, sensitive_feature, test_size=0.25, shuffle=True, stratify=train_y, random_state=42
    )
    
    train_x_tensor = torch.from_numpy(train_x).float()
    train_y_tensor = torch.from_numpy(train_y).float()
    sensitive_train_tensor = torch.from_numpy(sensitive_train).float()

    valid_x_tensor = torch.from_numpy(val_x).float()
    valid_y_tensor = torch.from_numpy(val_y).float()
    sensitive_val_tensor = torch.from_numpy(sensitive_val).float()

    # Create TensorDataset and DataLoader, including the sensitive attribute
    train_dataset = TensorDataset(train_x_tensor, train_y_tensor, sensitive_train_tensor)
    valid_dataset = TensorDataset(valid_x_tensor, valid_y_tensor, sensitive_val_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(valid_dataset, batch_size=batch_size)

    train_loaders.append(train_loader)
    val_loaders.append(val_loader)

# For test data
test_x = testset[feature_columns].values
test_y = testset['Two_yr_Recidivism'].values
sensitive_test = testset['caucasian'].values

test_x_tensor = torch.from_numpy(test_x).float()
test_y_tensor = torch.from_numpy(test_y).float()
sensitive_test_tensor = torch.from_numpy(sensitive_test).float()

test_dataset = TensorDataset(test_x_tensor, test_y_tensor, sensitive_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/usr/local/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (123) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (123) was not satisfied for alpha ([0.5]) after 1 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a des

## Client Model Architecture

In [11]:
class BaselineNN(nn.Module):
    def __init__(self):
        super(BaselineNN, self).__init__()
        self.fc1 = nn.Linear(5, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

def compute_eod(preds, labels, sensitive_feature):
    preds_binary = (preds >= 0.5).float()
    y_true_mask = (labels == 1).view(-1)

    p_a0 = preds_binary[y_true_mask & (sensitive_feature == 0)].mean().item()
    p_a1 = preds_binary[y_true_mask & (sensitive_feature == 1)].mean().item()

    eod = p_a0 - p_a1
    return eod

def train(net, trainloader, epochs, verbose=True):
    """
    Train Network on Training Set
    """
    criterion = nn.BCELoss()
    optimizer = optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        all_preds, all_labels, all_sensitives = [], [], []
        
        for inputs, labels, sensitive_features in trainloader:
            inputs, labels, sensitive_features = inputs.to(DEVICE), labels.to(DEVICE), sensitive_features.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            labels = labels.view(-1, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Append predictions and sensitive data for EOD computation
            all_preds.append(outputs.detach().cpu())
            all_labels.append(labels.detach().cpu())
            all_sensitives.append(sensitive_features.cpu())
        
        # Compute EOD at the end of the epoch
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        all_sensitives = torch.cat(all_sensitives)
        
        eod = compute_eod(all_preds, all_labels, all_sensitives)
        
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f} - EOD: {eod:.4f}")

def test(net, testloader, verbose=True):
    criterion = nn.BCELoss()
    net.eval()
    correct, total, loss = 0, 0, 0.0
    all_preds, all_labels, all_sensitives = [], [], []
    
    with torch.no_grad():
        for inputs, labels, sensitive_features in testloader:
            inputs, labels, sensitive_features = inputs.to(DEVICE), labels.to(DEVICE), sensitive_features.to(DEVICE)
            outputs = net(inputs)
            labels = labels.view(-1, 1)
            loss += criterion(outputs, labels).item() * inputs.size(0)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Append predictions and sensitive data for EOD computation
            all_preds.append(outputs.detach().cpu())
            all_labels.append(labels.detach().cpu())
            all_sensitives.append(sensitive_features.cpu())
    
    # Compute EOD at the end of testing
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    all_sensitives = torch.cat(all_sensitives)
    
    eod = compute_eod(all_preds, all_labels, all_sensitives)
    
    loss /= len(testloader.dataset)
    acc = correct / total
    if verbose:
        print(f"Test Loss: {loss:.4f} - Acc: {acc:.4f} - EOD: {eod:.4f}")
    return loss, acc, eod

# Centralized Learning

In [12]:
model = BaselineNN()

In [13]:
for i in range(NUM_CLIENTS):
    train_loader = train_loaders[i]
    val_loader = val_loaders[i]
    model = model.to(DEVICE)
    epochs = 10

    for epoch in range(epochs):
        train(model, train_loader, 1, verbose=False)
        loss, acc, eod = test(model, val_loader, verbose=False)

    loss, acc, eod = test(model, test_loader, verbose=False)
    print(f"Client {i} - Test Loss: {loss:.4f} - Acc: {acc:.4f} - EOD: {eod:.4f}")

Client 0 - Test Loss: 0.6444 - Acc: 0.6332 - EOD: 0.2178
Client 1 - Test Loss: 0.6374 - Acc: 0.6656 - EOD: 0.2322
Client 2 - Test Loss: 0.6191 - Acc: 0.6640 - EOD: 0.2690
Client 3 - Test Loss: 0.6267 - Acc: 0.6599 - EOD: 0.2376
Client 4 - Test Loss: 0.6196 - Acc: 0.6607 - EOD: 0.2596
Client 5 - Test Loss: 0.6325 - Acc: 0.6632 - EOD: 0.2707
Client 6 - Test Loss: 0.6496 - Acc: 0.6696 - EOD: 0.2587
Client 7 - Test Loss: 0.6229 - Acc: 0.6615 - EOD: 0.2485
Client 8 - Test Loss: 0.6231 - Acc: 0.6688 - EOD: 0.2724
Client 9 - Test Loss: 0.6424 - Acc: 0.6648 - EOD: 0.2163


# Federated Learning with Flower

In [14]:
import flwr as fl
from flwr.simulation import run_simulation
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents

from collections import OrderedDict
import numpy as np
import pandas as pd
import torch

from custom_flwr.server_app import server_fn as server_fn_custom
from custom_flwr.client_app import client_fn as client_fn_custom

DEVICE = torch.device('cpu')

def server_fn(context: Context):
    context.run_config = {
        'num-server-rounds' : 10,
        'fraction-fit': 1,
        'fraction-evaluate': 1,
        'local-epochs': 2,
        'server-device': str(DEVICE),
        'use-wandb': False
    }
    return server_fn_custom(context)

def client_fn(context: Context):
    return client_fn_custom(context)

client = ClientApp(client_fn=client_fn)
server = ServerApp(server_fn=server_fn)


backend_config = {"client_resources": None}
NUM_PARTITIONS = 10
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

/usr/local/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values.
  warnings.warn(
INFO :      Starting Flower ServerApp, config: num_rounds=10, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 23.615980350054226, {'centralized_accuracy': 0.5279352

Test Accuracy: 0.5279352226720648 - Test Loss: 23.615980350054226 - EOD: 0.0


INFO :      configure_fit: strategy sampled 2 clients (out of 10)
(ClientAppActor pid=12891) /usr/local/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values.
(ClientAppActor pid=12891)   warnings.warn(
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      fit progress: (1, 22.141467853998527, {'centralized_accuracy': 0.5417004048582996, 'eod': 0.00829804502427578}, 15.28975638600241)
INFO :      configure

(ClientAppActor pid=12891) Avg Train Loss: 0.6723190460886274 - EOD: nan - Accuracy: 0.6334841628959276
LOGG: RESULTS
{'acc': 0.5863192182410424, 'eod': nan, 'train_loss': 0.7348004758358002, 'id': 4} 307
{'acc': 0.6334841628959276, 'eod': nan, 'train_loss': 0.6723190460886274, 'id': 3} 442
============= LOGGG USING FEDAVG WEIGHTS
============= LOGGG WEIGHT RESULTS
[4, 3]
{4: 0.4098798397863818, 3: 0.5901201602136181}
Test Accuracy: 0.5417004048582996 - Test Loss: 22.141467853998527 - EOD: 0.00829804502427578
(ClientAppActor pid=12891) Test Accuracy: 0.5436893203883495 - Test Loss: 17.856506437063217 - EOD: 0.0625
(ClientAppActor pid=12890) Test Accuracy: 0.5526315789473685 - Test Loss: 22.00140569607417 - EOD: nan


(ClientAppActor pid=12888) /usr/local/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values. [repeated 24x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(ClientAppActor pid=12888)   warnings.warn( [repeated 24x across cluster]
INFO :      aggregate_evaluate: rece

...

(ClientAppActor pid=12888) /usr/local/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 6 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values. [repeated 352x across cluster]
(ClientAppActor pid=12888)   warnings.warn( [repeated 352x across cluster]
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      fit progress: (10, 19.433958238516098, {'centralized_accuracy': 0.6089068825910932, 'eod': 0.10440284013748169}, 35.09930319600244)
INFO :      configure_e

LOGG: RESULTS
{'acc': 0.6862745098039216, 'eod': 0.4085365831851959, 'train_loss': 0.611354511976242, 'id': 4} 306
{'acc': 0.6785714285714286, 'eod': -0.6181818246841431, 'train_loss': 0.5467515960335732, 'id': 9} 225
{'acc': 0.6948356807511737, 'eod': -0.190476194024086, 'train_loss': 0.633440511567252, 'id': 6} 213
{'acc': 0.6727272727272727, 'eod': 0.151324063539505, 'train_loss': 0.5931812723477682, 'id': 5} 165
{'acc': 0.6361607142857143, 'eod': nan, 'train_loss': 0.6585283960614886, 'id': 3} 448
{'acc': 0.6414473684210527, 'eod': 0.2720588147640228, 'train_loss': 0.6552415430545807, 'id': 2} 304
{'acc': 0.7160493827160493, 'eod': 0.3512931168079376, 'train_loss': 0.925036201874415, 'id': 8} 162
{'acc': 0.6794871794871795, 'eod': nan, 'train_loss': 0.6481518944104513, 'id': 1} 78
{'acc': 0.6096718480138169, 'eod': 0.10165733098983765, 'train_loss': 0.6560756752365514, 'id': 7} 579
{'acc': 0.7024647887323944, 'eod': nan, 'train_loss': 0.6247093048360612, 'id': 0} 568
============= 

INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 35.94s
INFO :      	History (loss, distributed):
INFO :      		round 1: 19.641226855209545
INFO :      		round 2: 19.14696844115269
INFO :      		round 3: 17.97194014636437
INFO :      		round 4: 18.90546002182633
INFO :      		round 5: 18.658285854781997
INFO :      		round 6: 18.12478473195878
INFO :      		round 7: 18.664356230037626
INFO :      		round 8: 17.50483912653218
INFO :      		round 9: 18.115507849147395
INFO :      		round 10: 17.20146466645263
INFO :      	History (loss, centralized):
INFO :      		round 0: 23.615980350054226
INFO :      		round 1: 22.141467853998527
INFO :      		round 2: 21.64446102655851
INFO :      		round 3: 20.01254916496766
INFO :      		round 4: 20.701260731770443
INFO :      		round 5: 19.7599449799611
INFO :      		round 6: 20.290802784455128
INFO :      		round 7: 20.98172941421851
INFO :      		round 8

(ClientAppActor pid=12889) Avg Train Loss: 0.925036201874415 - EOD: 0.3512931168079376 - Accuracy: 0.7160493827160493 [repeated 14x across cluster]
(ClientAppActor pid=12891) Test Accuracy: 0.6727941176470589 - Test Loss: 17.994021309746635 - EOD: -0.057887762784957886 [repeated 20x across cluster]
(ClientAppActor pid=12890) Test Accuracy: 0.6533333333333333 - Test Loss: 16.145010987917583 - EOD: nan [repeated 10x across cluster]
(ClientAppActor pid=12889) Avg Train Loss: 0.6481518944104513 - EOD: nan - Accuracy: 0.6794871794871795 [repeated 6x across cluster]
(ClientAppActor pid=12888) Skipping batch with single class.


(ClientAppActor pid=12890) /usr/local/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values. [repeated 45x across cluster]
(ClientAppActor pid=12890)   warnings.warn( [repeated 45x across cluster]
